In [1]:
from pymongo import MongoClient
from matplotlib import pyplot as plt
import pandas as pd
import altair as alt

ModuleNotFoundError: No module named 'pymongo'

In [ ]:
# Class-based API for querying our Blue Bike Mongo database
class BlueBikeMongoConnection:
    def __init__(self, db_name, collection_name):
        """ Constructor: establishes database connection,
            then counts and reports existing documents """
        client = MongoClient()
        db = client[db_name]
        self.collection = db[collection_name]
        self.docs = self.collection.count_documents({})
        print(f'Connection established with {db_name}.{collection_name} '
              f'({self.docs} documents)')
    
    def count_documents(self):
        """ Returns current count of database documents """
        return self.docs

    def find_one(self):
        """ Returns random document from database """
        return self.collection.find_one()
    
    def rides_per_month(self):
        """ Gets count of total, subscriber, and non-subscriber rides
            per month for every month in the database (1/2019 - 3/2024),
            as a DataFrame """
        pipeline = [
            {
                "$project": {
                    "year": "$start_time.year",
                    "month": "$start_time.month",
                    "subscribed": "$subscribed"
                }
            },
            {
                "$group": {
                    "_id": {
                        "year": "$year",
                        "month": "$month"
                    },
                    "total_rides": {"$sum": 1},
                    "sub_rides": {"$sum": {"$cond": [{"$eq": ["$subscribed", 1]}, 1, 0]}},
                    "non_sub_rides": {"$sum": {"$cond": [{"$eq": ["$subscribed", 0]}, 1, 0]}}
                }
            },
            {
                "$sort": {"_id.year": 1, "_id.month": 1}
            }
        ]

        agg = self.collection.aggregate(pipeline)
        data = []
        for idx, entry in enumerate(agg):
            year = entry['_id']['year']
            month = entry['_id']['month']
            num_rides = entry['total_rides']
            sub_rides = entry['sub_rides']
            non_sub_rides = entry['non_sub_rides']
            data.append({'id': idx, 'month': month, 'year': year,
                         'sub rides': sub_rides, 'non sub rides': non_sub_rides,
                         'number of rides': num_rides})
        return pd.DataFrame(data)

    def get_distinct(self, criterion):
        """ Customizable method to get distinct instances of a field """
        return self.collection.distinct(criterion)
    
    def get_municipalities(self):
        """ Gets distinct municipality instances """
        return self.collection.distinct('start_station.municipality')
    
    def seasonal_proportions_per_year(self):
        """ Gets count of total, subscriber, and non-subscriber rides
            per season for every season of each year, as a DataFrame """
        pipeline = [
            {
                "$project": {
                    "year": "$start_time.year",
                    "month": "$start_time.month",
                    "subscribed": "$subscribed"
                }
            },
            {
                "$group": {
                    "_id": {
                        "year": "$year",
                        "season": {
                            "$switch": {
                                "branches": [
                                    {"case": {"$in": ["$month", [3, 4, 5]]}, "then": "b_spring"},
                                    {"case": {"$in": ["$month", [6, 7, 8]]}, "then": "c_summer"},
                                    {"case": {"$in": ["$month", [9, 10, 11]]}, "then": "d_fall"},
                                    {"case": {"$in": ["$month", [12, 1, 2]]}, "then": "a_winter"}
                                ],
                                "default": "other"
                            }
                        }
                    },
                    "total_rides": {"$sum": 1},
                    "total_subscriber_rides": {"$sum": {"$cond": [{"$eq": ["$subscribed", 1]}, 1, 0]}},
                    "total_non_subscriber_rides": {"$sum": {"$cond": [{"$eq": ["$subscribed", 0]}, 1, 0]}}
                }
            },
            {
                "$group": {
                    "_id": {"year": "$_id.year", "season": "$_id.season"},
                    "average_total_rides": {"$avg": "$total_rides"},
                    "average_sub_rides": {"$avg": "$total_subscriber_rides"},
                    "average_non_sub_rides": {"$avg": "$total_non_subscriber_rides"}
                }
            },
            {
                "$sort": {"_id.year": 1, "_id.season": 1}
            }
        ]

        agg = self.collection.aggregate(pipeline)
        data = []
        for idx, entry in enumerate(agg):
            year = entry['_id']['year']
            season = entry['_id']['season']
            num_rides = entry['average_total_rides']
            sub_rides = entry['average_sub_rides']
            non_sub_rides = entry['average_non_sub_rides']
            data.append({'id': idx, 'season': season, 'year': year,
                         'avg sub rides': sub_rides, 'avg non sub rides': non_sub_rides,
                         'avg number of rides': num_rides})
        res = pd.DataFrame(data)
        res['season'] = res['season'].apply(lambda s: s.split("_")[1])
        return res
    
    def popular_routes(self):
        """ Gets most popular routes """
        pipeline = [
            {
                "$group": {
                    "_id": {
                        "start": "$start_station.name",
                        "end": "$end_station.name"
                    },
                    "num": { "$sum": 1 }
                }
            },
            {
                "$sort": { "num": -1 }
            }
        ]
        return self.collection.aggregate(pipeline)
    
    def circular_routes(self):
        """ Gets routes starting and ending at the same station """
        pipeline = [
            {
                "$match": {
                    "$expr": {
                        "$eq": ["$start_station.name", "$end_station.name"]
                    }
                }
            }
        ]
        return self.collection.aggregate(pipeline)
    
    def ending_in_boston(self):
        """ Gets routes ending at a station in the Boston municipality """
        pipeline = [
            {
                "$group": {
                    "_id": {
                        "start": "$start_station.municipality",
                        "end": "$end_station.municipality"
                    },
                    "num": { "$sum": 1 }
                }
            },
            {
                "$match": {
                    "$expr": {
                        "$eq": ["$_id.end", "Boston"]
                    }
                }
            },
            {
                "$sort": { "num": -1 }
            }
        ]

        return self.collection.aggregate(pipeline)
    
    def starting_in_boston(self):
        """ Gets routes starting at a station in the Boston municipality """
        pipeline = [
            {
                "$group": {
                    "_id": {
                        "start": "$start_station.municipality",
                        "end": "$end_station.municipality"
                    },
                    "num": { "$sum": 1 }
                }
            },
            {
                "$match": {
                    "$expr": {
                        "$eq": ["$_id.start", "Boston"]
                    }
                }
            },
            {
                "$sort": { "num": -1 }
            }
        ]

        return self.collection.aggregate(pipeline)

    def custom_pipeline(self, pipeline):
        """ Customizable pipeline aggregation method """
        return self.collection.aggregate(pipeline)
    
    def crosses_river(self):
        """ Gets routes that cross the Charles river
            based on their start and end station """
        pipeline = [
            {
                "$match": {
                    "$or": [
                        {
                            "$and": [
                                {"start_station.municipality": {"$in": ["Arlington", "Cambridge", "Chelsea", "Everett", "Malden", "Medford", "Revere", "Salem", "Somerville", "Watertown"]}},
                                {"end_station.municipality": {"$in": ["Boston", "Brookline", "Newton"]}}
                            ]
                        },
                        {
                            "$and": [
                                {"end_station.municipality": {"$in": ["Arlington", "Cambridge", "Chelsea", "Everett", "Malden", "Medford", "Revere", "Salem", "Somerville", "Watertown"]}},
                                {"start_station.municipality": {"$in": ["Boston", "Brookline", "Newton"]}}
                            ]
                        }
                    ]
                }
            },
            {
                "$group": {
                    "_id": {
                        "year": "$start_time.year",
                        "month": "$start_time.month"
                    },
                    "crosses_river_rides": { "$sum": 1 }
                }
            },
            {
                "$sort": {"_id.year": 1, "_id.month": 1}
            }
        ]

        agg = self.collection.aggregate(pipeline)
        data = []
        for idx, entry in enumerate(agg):
            year = entry['_id']['year']
            month = entry['_id']['month']
            rides = entry['crosses_river_rides']
            data.append({'id': idx, 'month': month, 'year': year,
                         'river-crossing rides': rides})
        
        return pd.DataFrame(data)
    
    def avg_duration_per_month(self):
        """ Gets the average duration of rides for subscribers
            and non-subscribers per month, for every month in the database
            (1/2019 - 3/2024), as a DataFrame """
        pipeline = [
            {
                "$group": {
                    "_id": {
                        "year": "$start_time.year",
                        "month": "$start_time.month",
                        "sub": "$subscribed"
                    },
                    "average_duration": { "$avg": "$duration" }
                }
            }
        ]

        agg = self.collection.aggregate(pipeline)
        sub, nonsub = [], []
        for idx, entry in enumerate(agg):
            month = entry["_id"]["month"]
            year = entry["_id"]["year"]
            subbed = True if entry["_id"]["sub"] == 1 else False
            duration = entry["average_duration"]
            if subbed:
                sub.append({'id': idx, 'month': month, 'year': year,
                            'avg duration': duration})
            else:
                nonsub.append({'id': idx, 'month': month, 'year': year,
                               'avg duration': duration})
        
        sub.sort(key=lambda i: (i['year'], i['month']))
        nonsub.sort(key=lambda i: (i['year'], i['month']))
        return pd.DataFrame(sub).reset_index(), pd.DataFrame(nonsub).reset_index()

In [ ]:
# Set personal database name and collection name
# Then create BlueBikeMongoConnection instance
db_name = 'blueBikes'
collection_name = 'trip'
conn = BlueBikeMongoConnection(db_name, collection_name)

Connection established with DS4300.bluebikes (15622370 documents)


In [ ]:
# Plot seasonal_proportions_per_year() data
df = conn.seasonal_proportions_per_year()
domain = ['fall', 'spring', 'summer', 'winter']
_range = ['#FC8F36', '#23BE61', '#FFCB47', '#33CFFF']

axis_labels = ("datum.label == '999' ? 'Spring' : '' \
               ")

chart = alt.Chart(df).mark_bar().encode(
            y = alt.Y("avg number of rides:Q", title="Average Rides"),
            x =alt.X("id", axis=alt.Axis(title="Season", labelExpr=axis_labels)).sort("y"),
            color=alt.Color("season:N").scale(domain=domain, range=_range)
        ).properties(
            title = {"text": "Average Monthly Rides per Season",
                    "subtitle": "01/2019 - 03/2024"}
        )

chart.show()

alt.Chart(...)

In [ ]:
# Plot rides_per_month() data
df_2 = conn.rides_per_month()
sub_chart = alt.Chart(df_2).mark_line().encode(x=alt.X('id', title=""),
                                 y=alt.Y("sub rides", title=""),
                                 color=alt.value("#33CFFF"))
non_sub_chart = alt.Chart(df_2).mark_line().encode(x=alt.X("id",
                                                           axis=alt.Axis(labels=False,
                                                                         title="Month")),
                                     y=alt.Y("non sub rides", title="Rides"),
                                     color=alt.value("#FC8F36"))
chart = alt.layer(sub_chart,non_sub_chart).properties(title="Rider Subscription Status per Month")
chart

alt.LayerChart(...)

In [ ]:
sub_dur_df, nonsub_dur_df = conn.avg_duration_per_month()
sub_dur_chart = alt.Chart(sub_dur_df).mark_line().encode(
    x=alt.X("index", axis=alt.Axis(labels=False, title="Month")),
    y=alt.Y("avg duration", title=""),
    color=alt.value("#33CFFF")
)
nonsub_dur_chart = alt.Chart(nonsub_dur_df).mark_line().encode(
    x=alt.X("index"), y=alt.Y("avg duration", title="Average Duration (seconds)"),
    color=alt.value("#FC8F36")
)

chart = alt.layer(sub_dur_chart, nonsub_dur_chart).properties(title="Ride Durations per Month")
chart

alt.LayerChart(...)

In [ ]:
eletric_v_classic_2 = {"labels": ["Total Duration", "# of Rides", "# Cross-Muncipality", "Fleet Presence"],
                    "data": [20.34076086, 25.69639499, 27.9048742, 15.78947368]}

df_e = pd.DataFrame(eletric_v_classic_2)

domain = ["Total Duration", "# of Rides", "# Cross-Muncipality", "Fleet Presence"]
_range = ["#4285f4", "#4285f4", "#4285f4", '#ffab40']

base = alt.Chart(df_e).mark_bar(size=45).encode(
    x = alt.X("labels:N", axis=alt.Axis(labelAngle=0, title="Categories")).sort("y"),
    y = alt.Y("data:Q", axis=alt.Axis(title="% Accounted for by Electric Bikes"), scale=alt.Scale(domain=[0, 29])),
    color = alt.Color("labels:N", legend=None).scale(domain=domain, range=_range)
).properties(
    title={"text": "Electric Bikes Outperforming Fleet Presence",
           "subtitle": "12/2023 - 03/2024"},
    width=320
)

base